In [1]:
import numpy as np
from fmm_pcm import fmm_pcm

In [2]:
# COMPLEX
lmax = 2
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([.0, 0, 0.5])
sph2_r = 1
src1 = np.array([0.0, 0.5, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_complex(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_complex(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_complex_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_complex_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)

0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214
0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214


In [3]:
# REAL
lmax = 17
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([1.0, 1, 1])
sph2_r = 1
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_real(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_real(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_real_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_real_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)
#coef2_m_m2m = fmm_pcm.m2m_complex_translate_z(sph1_c[2]-sph2_c[2], lmax, coef1_m)
#print(coef1_m)
#print(coef2_m)
#print(coef2_m_m2m)
#print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_m2m), np.linalg.norm(src1-dst1)**-1)

0.3254722774521031 0.32547227745205964
0.32525817347924185 0.32547227745205964
0.3254722774521031 0.32547227745205964
0.3252581734792418 0.32547227745205964


In [4]:
# Normalized real spherical harmonics
lmax = 60
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1.5
sph2_c = np.array([1.0, 1, 1])
sph2_r = 2*3**0.5 + 1.5
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2o(src1-sph1_c, sph1_r, lmax)
coef2_m = fmm_pcm.p2o(src1-sph2_c, sph2_r, lmax)
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)

0.3254722774520597 0.32547227745205964
0.3254718954621216 0.32547227745205964


In [5]:
# Compute M2M matrix by using P2M and M2P
lmax = 20
nparticles = 2000
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 0.020
sph2_c = np.array([0.10, 0.020, 0.30], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c)
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.complex128)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.complex128)
np.random.seed(0)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2m_complex(particles[i]-sph1_c, lmax)
    mat2[i] = fmm_pcm.p2m_complex(particles[i]-sph2_c, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
#print(m2m)
print(np.max(m2m[1:]), np.min(m2m))
mat3 = fmm_pcm.m2m_baseline(sph1_c-sph2_c, lmax, mat1[0])
#print(m2m[10,0])
#print(mat1[0][:10])
#print(mat2[0][:10])
#print(mat3[:10])
print(np.linalg.norm(mat3-mat2[0]) / np.linalg.norm(mat2[0]))
#print(np.linalg.norm(m2m, ord=np.inf))
print(np.max(np.linalg.norm(m2m, axis=1)))

0.00440888762644122
(4.6457468943419874e+23-1.774461331471828e+23j) (-1.962162231360827e+24+1.9458403204634595e+23j)
1.9165935371633602e-17
6.730265436955804e+24


In [6]:
# Compute O2O matrix by using P2O and O2P
lmax = 20
nparticles = 2000
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 20
sph2_c = np.array([10, 20, 30], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c)
#sph2_r = 40
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
np.random.seed(0)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2o(particles[i]-sph1_c, sph1_r, lmax)
    mat2[i] = fmm_pcm.p2o(particles[i]-sph2_c, sph2_r, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
#print(m2m)
print(np.max(m2m[1:]), np.min(m2m))
mat3 = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, mat1[0])
#print(m2m[10,0])
#print(mat1[0])
#print(mat2[0])
#print(mat3)
print(np.linalg.norm(mat3-mat2[0]) / np.linalg.norm(mat2[0]))
mat4 = fmm_pcm.o2o_matrix(sph1_c-sph2_c, sph1_r, sph2_r, lmax) @ mat1[0]
print(np.linalg.norm(mat4-mat3))

5.3804368179792985e-14
0.6033278166540186 -0.9049917249810278
5.192585567995216e-16
6.49002077467652e-17


In [7]:
# CHECK O2O (scaled M2M)
lmax = 20
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([1, 0.5, 0.5], dtype=np.float64)
sph2_r = 4
dst1 = np.array([4.2, 0., 2.])
coef1_m = np.random.randn((lmax+1)**2)
coef2_m = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, coef1_m)
coef3_m = fmm_pcm.o2o_matrix(sph1_c-sph2_c, sph1_r, sph2_r, lmax) @ coef1_m
print(np.linalg.norm(coef2_m-coef3_m)/np.linalg.norm(coef2_m))
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m),
     fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef3_m))
#print(coef2_m, "\n", coef3_m, "\n", coef2_m-coef3_m)
#print(coef1_m)

1.1627451061300556e-16
1.4715870995839442 1.4732865861787003 1.4732865861787001


In [8]:
# Check O2O matrix
lmax = 20
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 0.1
sph2_c = np.array([1, 0.5, 0.2], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c)
sph3_c = np.array([0, 0, 0], dtype=np.float64)
sph3_r = 20000
sph4_c = np.array([100000, 50000, 20000], dtype=np.float64)
sph4_r = sph3_r + np.linalg.norm(sph4_c)
o2o_mat1 = fmm_pcm.o2o_matrix(sph1_c-sph2_c, sph1_r, sph2_r, lmax)
o2o_mat2 = fmm_pcm.o2o_matrix(sph3_c-sph4_c, sph3_r, sph4_r, lmax)
print(np.linalg.norm(o2o_mat1-o2o_mat2)/np.linalg.norm(o2o_mat1))
#print(np.linalg.norm(o2o_mat2, ord=np.inf))
print(np.max(np.linalg.norm(o2o_mat1, axis=1)))
#print(np.linalg.svd(o2o_mat1)[1])

0.49152651110296247
1.5428307405046444


In [1]:
import numpy as np
from fmm_pcm import fmm_pcm
# Check geometry division
sph = np.random.randn(3, 4)
ind = np.arange(1, sph.shape[1]+1, dtype=np.int32)
print(ind)
div = fmm_pcm.geometry_divide(sph, ind)
print(div, ind)
#print(sph[:, 0], sph[:, 1])

[1 2 3 4]
3 [1 2 3 4]


In [7]:
x = np.array([-0.70584076305585219, -0.34962505235150587, 0.61607721916732117])
y = np.array([-8.0257977751952438, -3.9754291820252918, 7.0051369001910491])
11.370550123017157*x-y

array([ 0.00000000e+00, -3.10862447e-15,  2.66453526e-15])